# ASTR302 Lab 7: Astrometry and Calibrated Photometry

In this Lab you will determine the right ascension and declination of your sources and calibrate the photometry

## Where are we?

You probably pointed the telescope at this field, or at least someone did. So you would think we would know the coordinates. However, in general the telescope pointing is not succiently precise to serve our purposes. We want to do better. 

Before we start, lets import the packages you will be needing for this Lab. You'll be using the astrometry solver which is based on the astronomy.net algorithm. 

In [ ]:
#lets install the astrometry package (from astronomy.net)
!pip install --upgrade pip
!pip install astrometry

import matplotlib.pyplot as plt
import csv
import astrometry
import pandas as pd
import astropy.units as u
import numpy as np

import astropy as ap
from astropy.wcs import WCS
from astropy.io import fits

# the 'scales' in the following need to be chosen match to the estimated field-of-view (see https://pypi.org/project/astrometry/)
# these scales are the ones for the image you are working with

solver = astrometry.Solver(
    astrometry.series_5200.index_files(
        cache_directory="astrometry_cache",
        scales={1,3},
    )
)

This next cell just reads in your photometry file from before and gets the list of stars that will be used by the astrometry solver.

In [ ]:
# read the CSV file and make a list of coordinate pairs for astronometry solver
import re
catalog = pd.read_csv('photometry.csv')

unit = "pix"
x = [sub.replace(unit, "").strip() for sub in catalog['xcenter']]
y = [sub.replace(unit, "").strip() for sub in catalog['ycenter']]

stars = [(x[i],y[i]) for i in range(0,len(x))]

Now on to the actual solver. At the end we print out the header coordinates and the solved coordinates for comparison.

In [ ]:
filename = 'imacs_image.fits'
hdu = fits.open(filename)[0]
image = hdu.data
hdr = hdu.header

import astrometry
import logging

logging.getLogger().setLevel(logging.INFO)

# start with values close to the what is given in the header
solution = solver.solve(
    stars=stars,
    size_hint=None,
    position_hint=astrometry.PositionHint(
        ra_deg=346.2,
        dec_deg=-8.68,
        radius_deg=1.0),
    solution_parameters=astrometry.SolutionParameters(),
)    

# defines new wcs reference frame using solved for values

if solution.has_match():
    wcs = solution.best_match().astropy_wcs()
    
# it there is a good solution check out correspondence with previous values
    print('The header RA is ',hdr['RA-D'],'and the solved RA is ',solution.best_match().center_ra_deg)
    print('The header Dec is ',hdr['DEC-D'],'and the solved Dec is ',solution.best_match().center_dec_deg)
    print('The header plate scale is ',hdr['SCALE'],'and the solved plate scale is ',solution.best_match().scale_arcsec_per_pixel)

Now we will display the image, with the new coordinates, and proceed to list the coordinates of your stars. The way we will calibrate the photometry is by comparing the instrumental magnitudes you have measured for your stars with the available Sloan Digital Sky Survey values available through the Legacy Viewer (https://www.legacysurvey.org/viewer/). To keep this relatively simple, we'll do this by hand. Note that the way the image is displayed here is flipped relative to how the Viewer shows the field. You will also need to adjust the vmin and vmax values to get the best image display.

In [ ]:
# define the image, open, and read the header information - use your reduced image from the previous workbook

# Create the plotting object with the WCS projection.
plt.figure(figsize = (20,20))
plt.subplot(projection=wcs)
plt.imshow(image, vmin=1140, vmax=1220)
plt.grid(color='white', ls='solid', alpha=1)
plt.xlabel('Right Ascension',fontsize=24)
plt.ylabel('Declination',fontsize=24)

In [ ]:
# print out the coordinates of the stars that were used for the coordinate solution

if solution.has_match():
    for star in solution.best_match().stars:
        print(f"{star.ra_deg}, {star.dec_deg}:")

You have a choice here. You can just use the Legacy Viewer and the coordinates of these stars to get the SDSS magnitudes (use r band, but also keep the g band as we will want to look at the dependence of the calibration vs. color) to build up your photometric calibration or you can use astroquery (https://iopscience.iop.org/article/10.3847/1538-3881/aafc33) to get the magnitudes from the SIMBAD database. The latter will likely require much more effort, but will be more general if you want to make it so.

<div class="alert alert-info"> Make a plot of instrumental magnitude vs. SDSS magnitude for these stars. Perform a linear fit to get the zero point (the offset) and check for linearity. </div>

Answer: 

<div class="alert alert-info"> Now plot the offset from the fit for each star vs. color. Do you see a trend or just scatter. If there is a trend, describe the possible origin and how to address it in the calibration.</div>

## Conclusion: 

 <div class="alert alert-info">Save your notebook.  Append your LastNameFirstInitial to the filename and submit via D2L </div>